In [14]:
!pip install emoji


In [15]:
import requests
from requests import session
import emoji
import json
s = session()
access_token = 'EAAD6V7os0gcBOwYNmlpf1MlFl8YHpocZC5nZCTeUmnflBVtWtxPcq1hw6ZAsvdNwnadZCJWQdhF9A33lo2K03KOn8JZBrcHKKIrJ4yn66pazz75eomaNbYv5ZAcx7bqzSgboZAgxQYQ9ELepPzXqAbn64aF9SFSxFBR3ZASuLofAGQiDYqGDHW1zvXoJZCFuQSVxnPMeZBmZA3FnwZDZD'
id_fanpage = '100067615970201'
name_fanpage = 'anhdavangvuive' # Fanpage: Anh Da Vàng
fields_feed = 'feed' # lấy tất cả các feed

In [16]:


raw_data_feed = s.get('https://graph.facebook.com/'+id_fanpage+'?fields='+fields_feed+'&access_token='+access_token) # có thể xem chi tiết về số lượng share, chi tiết comment, nội dung bài viết(message), thời gian đăng bài
#raw_data_feed.json()['feed']['data'][0]['created_time'][:10]\
#raw_data_feed.json()

In [17]:
#chạy lần đầu tiên lấy <= 25 feed đầu tiên cho vào list res_raw_json
count_feed = 0 # biến đếm số bài viết

res_raw_json = [] #list các feed (các feed ở dạng json)
for i in range(len(raw_data_feed.json()['feed']['data'])):
    count_feed += 1
    res_raw_json.append(raw_data_feed.json()['feed']['data'][i])
#chạy lần thứ 2 lấy <= 25 feed tiếp theo cho vào list res_raw_json    
mask_data_feed =  s.get(raw_data_feed.json()['feed']['paging']['next'])# gọi api đến next page (mỗi lần (default nếu không limit với giá trị limit có thể chạy từ 0 đến 250 bài) gọi chỉ lấy được 25 feed)  
for i in range(len(mask_data_feed.json()['data'])):
    count_feed += 1
    res_raw_json.append(mask_data_feed.json()['data'][i])
#tiếp tục chạy đến khi đủ 500 bài (page này đến thời điểm 3/11/2023 có hơn 5500 bài viết)
check_not_enough_feed = True # biến check đến khi đủ 500 bài 
while(check_not_enough_feed): 
    if('paging' in mask_data_feed.json()):
        mask_data_feed = s.get(mask_data_feed.json()['paging']['next'])
        if('data' in mask_data_feed.json()):
            for i in range(len(mask_data_feed.json()['data'])):
                count_feed += 1
                res_raw_json.append(mask_data_feed.json()['data'][i])
                if(count_feed >= 500):
                    check_not_enough_feed = False
                    break
with open('feed_raw_data.json', 'w') as feed_raw_data:
    json.dump(res_raw_json,feed_raw_data)
        

In [18]:
print(len(res_raw_json))

500


In [19]:
#hàm cleanString
def clean_string(st):
    e = emoji.emoji_list(emoji.emojize(st)) # lấy ra các emoji
    for i in e :
        st = st.replace(i['emoji'],'')      
    st =  st.replace("Cre:", '')    
    st =  st.replace("cre:", '')
    st =  st.replace("P/s:", '')
    st =  st.replace("p/s:", '')
    st =  st.replace("Cre :", '')
    st =  st.replace("cre :", '')
    st =  st.replace("cre", '')
    st =  st.replace("Cre", '')
    return st

In [20]:
#hàm tách hastag:
def split_hastag(s):
    start = s.find('#')
    result = s[start:]
    end  = result.find(' ')
    if end == -1:
        end  = result[1:].find('#')
    if end == -1:
        end = result.find('\n')
    if end == -1:
        return result[1:]    
    result = result[:end + 1]
    subresult = result[1:]
    subresult.replace("\n",'')
    subresult = subresult.strip()
    return subresult


In [23]:
#đọc file feed_raw_data để xử lí
raw_data_read = ''
with open('feed_raw_data.json','r') as handler:
    raw_data_read = json.load(handler)
#raw_data_read[0]['comments']['data'][0]['message_tags'][0]['id']    
comments_details_lists = [] #
list_message_post = [] #nội dung của các bài viết
list_time_created = [] #thời gian đăng bài 
hours_time_created = []#thời gian đăng bài theo giờ
shared_total = []#số lượng share 
source_picture = [] #list_link_picture
source_video = [] # list_link_video
comments_time_created = [] #list_time_created_comments
link_post = []#list_link_post
list_len_video =[]#list_len_of_video
list_hastag =[] # list_hastag 
for post in raw_data_read:
    comments_details_lists_per_post = []
    #lấy nội dung bài viết(thay icon bằng chuỗi 'icon)
    raw_message_post = post['message']
    clean_message_post = clean_string(raw_message_post)
    for i in range(clean_message_post.count('#')):
        list_hastag.append(split_hastag(clean_message_post))
        sub = '#'+split_hastag(clean_message_post)
        clean_message_post= clean_message_post.replace(sub,'')
    list_message_post.append(clean_message_post)
    #lấy thời gian đăng bài
    list_time_created.append(post['created_time'])
    hours_time_created.append(post['created_time'][11:13])
    #lấy độ dài video (nếu có)
    len_video = '0'
    if('properties' in post):
         for vid in post['properties']:
              if('text'in vid):
                   len_video = vid['text']
    list_len_video.append(len_video)
    #lấy comment (25 comment gần nhất)
    comments_details_lists_per_post = []
    for cm in range(len(post['comments']['data'])):
            comments_details =  post['comments']['data'][cm]['from'] # user comments
            comments_details['message'] =  clean_string(post['comments']['data'][cm]['message']) # message comments
            comments_details['time_created'] = post['comments']['data'][cm]['created_time'][11:13]#time comments
            comments_details['id_tag'] = '' # user_tag 
            if('message_tags' in  post['comments']['data'][cm]):
                comments_details['id_tag'] = post['comments']['data'][cm]['message_tags'][0]['id']
            comments_details_lists_per_post.append(comments_details)
    comments_details_lists.append(comments_details_lists_per_post)        
    # đoạn lấy tất cả comment này dùng hơi tốn clone fb và thường bị chặn ip khi spam lấy quá nhiều comment (của 500 bài viết)   
        #lấy comment lần 2
    link_next_comments_page =''    
    if('next' in post['comments']['paging']):
            link_next_comments_page = s.get(post['comments']['paging']['next'] )
            #lấy dữ liệu
            if('data' in link_next_comments_page.json()):
                for cmt_d in link_next_comments_page.json()['data']:
                    cmt_dt = cmt_d['from']
                    cmt_dt['message'] = clean_string(cmt_d['message'])
                    cmt_dt['time_created'] = cmt_d['created_time'][11:13]
                    cmt_dt['id_tag'] = ''
                    if('message_tags' in cmt_d):
                        cmt_dt['id_tag'] = cmt_d['message_tags'][0]['id']
                    comments_details_lists_per_post.append(cmt_dt) 
                comments_details_lists.append(comments_details_lists_per_post)    
                     
            #     
    if(link_next_comments_page!= '') :       #   
        if('paging' in link_next_comments_page.json()):
            while ('next' in link_next_comments_page.json()['paging']):
                    link_next_comments_page = s.get(link_next_comments_page.json()['paging']['next'] )
                    #lấy dữ liệu
                    if('data' in link_next_comments_page.json()):  
                        for cmt_d in link_next_comments_page.json()['data']:
                            cmt_dt = cmt_d['from']
                            cmt_dt['message'] = clean_string(cmt_d['message'])
                            cmt_dt['time_created'] = cmt_d['created_time'][11:13]#checkpoint
                            cmt_dt['id_tag'] = ''
                            if('message_tags' in cmt_d):
                                cmt_dt['id_tag'] = cmt_d['message_tags'][0]['id']
                            comments_details_lists_per_post.append(cmt_dt)  
                        comments_details_lists.append(comments_details_lists_per_post)     
                               
                    else:
                        break         
                    #
       
    #lấy số lượng share
    if('shares' in post):
        shared_total.append(post['shares']['count'])       
    else:
         shared_total.append(0)
    #lấy source picture
    if('picture' in post):
         source_picture.append(post['picture'])
    else:
         source_picture.append('')     
    #lấy source video
    if('source' in post):
         source_video.append(post['source'])
    else:
         source_video.append('')    
    #lấy link post
    link = 'facebook.com/'+post['id']       
    link_post.append(link)     

# list_message_post
# list_time_created
# hours_time_created 
# comments_details_lists
# shared_total

In [24]:
#lưu list_hastag vào file_hastag.txt
with open('file_hastag.txt','w',encoding='utf8') as handler:
    for i in list_hastag:
        handler.write(i)
        handler.write("\n")

In [25]:
#lưu list_len_video vào file_len_video
with open('file_len_video.txt','w') as handler:
    for i in list_len_video:
        handler.write(i)
        handler.write("\n")

In [26]:
#lưu source_picture vào file file_source_picture
with open("file_source_picture",'w') as handler:
    for i in source_picture:
        handler.write(i)
        handler.write("\n")

In [27]:
#lưu source_video vào file file_source_video
with open("file_source_video",'w') as handler:
    for i in source_video:
        handler.write(i)
        handler.write("\n")

In [28]:
#lưu link_post vào file link_post.txt
with open("link_post.txt", 'w') as handler:
    for i in link_post:
        handler.write(i)
        handler.write("\n")

In [29]:
#lưu thời gian đăng bài theo giờ vào file hours_created.txt
with open("hours_created",'w') as handler:
    for i in hours_time_created:
        handler.write(i)
        handler.write("\n")

In [30]:
#lưu thời gian đăng bài vào file time_created.txt
with open("time_created", 'w') as handler:
    for i in list_time_created:
        handler.write(i)
        handler.write("\n")

In [31]:
#lưu nội dung bài viết vào file raw_message.txt
with open('raw_message.txt','w',encoding='utf8') as handler:
    for i in list_message_post:
        handler.write(i)
        handler.write("\n")
        handler.write("\n")

In [32]:
#lưu comments_details_lists vào file raw_comments.json
with open('raw_comments.json','w') as file_raw_comments:
    json.dump(comments_details_lists,file_raw_comments)


In [33]:
#lưu shared_total vào file_shared_total.txt
with open("file_shared_total.txt",'w') as handler:
    for i in shared_total:
        handler.write(str(i))
        handler.write("\n")

In [34]:
# lấy chi tiết từng reaction của 1 bài viết" LIKE, LOVE, WOW, HAHA, SAD, ANGRY, THANKFUL, PRIDE, CARE, FIRE, HUNDRED"
'100067615970201/feed?fields=reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow)'
raw_data_reaction = []
count_data_reactions = 0
fields_reactions_details = 'feed{reactions.type(LIKE).limit(0).summary(total_count).as(reactions_like),reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow),reactions.type(HAHA).limit(0).summary(total_count).as(reactions_haha),reactions.type(SAD).limit(0).summary(total_count).as(reactions_sad),reactions.type(ANGRY).limit(0).summary(total_count).as(reactions_angry)}' 
#chạy lần đầu tiên
raw_data_reaction_details = s.get('https://graph.facebook.com//v18.0/'+id_fanpage+'?fields='+fields_reactions_details+'&access_token='+access_token)
for i in raw_data_reaction_details.json()['feed']['data']:
    raw_data_reaction.append(i)
    count_data_reactions += 1
#

In [35]:
#chạy lần 2
next_page_reactions_details = ''
if('paging' in raw_data_reaction_details.json()['feed']):
    if('next' in raw_data_reaction_details.json()['feed']['paging']):
        next_page_reactions_details =  s.get(raw_data_reaction_details.json()['feed']['paging']['next'])
        #lấy reactions
        if('data' in next_page_reactions_details.json()):
            for i in next_page_reactions_details.json()["data"]:
                raw_data_reaction.append(i)
                count_data_reactions+=1
        #        
#                

In [36]:
#chạy để lấy tất cả reactions của 500 bài viết
check_not_enough_reactions = True
while('paging' in next_page_reactions_details.json() and check_not_enough_reactions):
    if('next' in next_page_reactions_details.json()['paging']):
        next_page_reactions_details =  s.get(raw_data_reaction_details.json()['feed']['paging']['next'])
        #lấy reactions
        if('data' in next_page_reactions_details.json()):
            for i in next_page_reactions_details.json()["data"]:
                raw_data_reaction.append(i)
                count_data_reactions+=1
                if count_data_reactions == 500 :
                    check_not_enough_reactions = False
                    break
        #          

In [37]:
#lưu raw_data_reaction vào file_raw_data_reactions.json
with open("file_raw_data_reactions.json",'w') as handler:
    json.dump(raw_data_reaction, handler)

In [38]:
total_reactions = []
#lấy tổng số reactions của từng bài viết
for j in range(len(raw_data_reaction)):
    total = 0
    for i in raw_data_reaction[j]:
        if('summary' in raw_data_reaction[j][i]):
            total += raw_data_reaction[j][i]['summary']['total_count']
    total_reactions.append(total)
        

In [39]:
#lưu total reactions vào file 
with open("file_total_reactions.txt",'w') as handler:
    for i in total_reactions:
        handler.write(str(i))
        handler.write("\n")

      

In [41]:
# list_message_post
# list_time_created
# hours_time_created 
# total_reactions
# shared_total
# comments_details_lists
# user_id

In [40]:
# lấy total comments của bài viết
fields_total_comments = 'feed{comments.summary(true)}'
count_total_feed_comments = 0
total_comments = []
#lần
raw_total_comments = s.get('https://graph.facebook.com/' + id_fanpage + '?' + 'fields=' + fields_total_comments+'&access_token='+access_token)
for i in range(len(raw_total_comments.json()['feed']['data'])):
    total_comments.append(raw_total_comments.json()['feed']['data'][i]['comments']['summary']['total_count'])
    count_total_feed_comments += 1
#lần 2
next_raw_total_comments = ''
if('paging' in raw_total_comments.json()['feed']):
    if('next' in raw_total_comments.json()['feed']['paging']):
        next_raw_total_comments= s.get(raw_total_comments.json()['feed']['paging']['next'])
        for i in range(len(next_raw_total_comments.json()['data'])):
            total_comments.append(next_raw_total_comments.json()['data'][i]['comments']['summary']['total_count'])  
            count_total_feed_comments += 1  
#lấy tiếp  
check_not_enough_feed_comments = True
while(check_not_enough_feed_comments):
    if('paging' in next_raw_total_comments.json()):
        if('next' in next_raw_total_comments.json()['paging']):
            next_raw_total_comments= s.get(next_raw_total_comments.json()['paging']['next'])
            for i in range(len(next_raw_total_comments.json()['data'])):
                total_comments.append(next_raw_total_comments.json()['data'][i]['comments']['summary']['total_count'])  
                count_total_feed_comments += 1  
                if(count_total_feed_comments == 500):
                    check_not_enough_feed_comments = False
                    break            


In [42]:
# lưu tổng số comments của từng bài viết  vào file
with open('total_comments.txt','w') as handler:
    for i in total_comments:
        handler.write(str(i))
        handler.write("\n")
                        

In [43]:
# list_message_post
# list_time_created
# hours_time_created 
# total_comments
# total_reactions
# raw_data_reaction
# shared_total
# comments_details_lists
# user_id
# source picture
# source video

In [44]:
total_reactions_like = []
total_reactions_love = []
total_reactions_wow = []
total_reactions_haha = []
total_reactions_sad = []
total_reactions_angry = []
for i in raw_data_reaction:
    
    total_reactions_like.append(i['reactions_like']['summary']['total_count'])
    total_reactions_love.append(i['reactions_love']['summary']['total_count'])
    total_reactions_wow.append(i['reactions_wow']['summary']['total_count'])
    total_reactions_haha.append(i['reactions_haha']['summary']['total_count'])
    total_reactions_sad.append(i['reactions_sad']['summary']['total_count'])
    total_reactions_angry.append(i['reactions_angry']['summary']['total_count'])

{'feed': {'data': [{'reactions_like': {'data': [],
     'summary': {'total_count': 1046}},
    'reactions_love': {'data': [], 'summary': {'total_count': 7}},
    'reactions_wow': {'data': [], 'summary': {'total_count': 7}},
    'reactions_haha': {'data': [], 'summary': {'total_count': 1936}},
    'reactions_sad': {'data': [], 'summary': {'total_count': 23}},
    'reactions_angry': {'data': [], 'summary': {'total_count': 0}},
    'reactions_thankfull': {'data': [], 'summary': {'total_count': 0}},
    'reactions_pride': {'data': [], 'summary': {'total_count': 0}},
    'reactions_care': {'data': [], 'summary': {'total_count': 0}},
    'reactions_fire': {'data': [], 'summary': {'total_count': 0}},
    'reactions_hundred': {'data': [], 'summary': {'total_count': 0}},
    'id': '1521822118121953_674869951443531'},
   {'reactions_like': {'data': [], 'summary': {'total_count': 4879}},
    'reactions_love': {'data': [], 'summary': {'total_count': 30}},
    'reactions_wow': {'data': [], 'summary

In [45]:
#lấy tất cả các user id của bình luận cho vào set (không trùng lặp người bình luận)
user_id_comment_set =  set()
user_id_comment_list = []
user_id_tag_list = []
for i in comments_details_lists:
    for j in i:
        user_id_comment_set.add(j['id'])
        user_id_comment_list.append(j['id'])
        if(j['id_tag'] != ''):
            user_id_tag_list.append(j['id_tag'])
#lấy tất cả user id của bình luận (có tính trùng lặp)
#lấy tất cả id_tag (id người được tag trong bình luận)

In [46]:
print(len(user_id_comment_list))
print(len(user_id_comment_set))
print(len(user_id_tag_list))
#hơn 1 nửa số comments là có tag nhau 

11896
9973
6873


In [47]:
#lưu user_id_comment_set vào trong file.txt
with open('file_user_id_comment_set.txt', 'w') as handler:
    for i in user_id_comment_set:
        handler.write(str(i))
        handler.write("\n")
#lưu user_id_comment_list vào file.txt
with open('file_user_id_comment_list.txt', 'w') as handler:
    for i in user_id_comment_list:
        handler.write(str(i))
        handler.write("\n")
#lưu user_id_tag_list vào file.txt
with open('file_user_id_tag_list.txt', 'w') as handler:
    for i in user_id_tag_list:
        handler.write(str(i))
        handler.write("\n")        

In [48]:
#cho dữ liệu vào file table_data
big_table = {'message_post' : list_message_post, 'id_post' : link_post,'time_created_total' : list_time_created, 'time_created_hours' : hours_time_created, 
                          'total_of_comments' : total_comments, 'total_of_reactions' : total_reactions, 'reactions_like' : total_reactions_like,'reactions_love' : total_reactions_love,
                          'reactions_wow' : total_reactions_wow, 'reactions_haha' : total_reactions_haha, 'reactions_sad' : total_reactions_sad, 'reactions_angry' : total_reactions_angry,
                          'total_of_share':shared_total, 'comments_details' : comments_details_lists, 'source_of_picture': source_picture, 
                          'source_of_video' : source_video, 'len_video' : list_len_video}


In [49]:
data_table = []
for i in range(500):
    data_table.append({'message_post': big_table['message_post'][i], 'id_post': big_table['id_post'][i], 'time_created' : big_table['time_created_total'][i],
                       'hours_time_created' : big_table['time_created_hours'][i], 'total_comments': big_table['total_of_comments'][i],
                       'total_of_reactions': big_table['total_of_reactions'][i],
                       'reactions_like': big_table['reactions_like'][i], 'reactions_love': big_table['reactions_love'][i],
                       'reactions_wow': big_table['reactions_wow'][i],
                       'reactions_haha': big_table['reactions_haha'][i],
                       'reactions_sad': big_table['reactions_sad'][i],
                       'reactions_angry': big_table['reactions_angry'][i],'total_share': big_table['total_of_share'][i],
                       'comments_details' : big_table['comments_details'][i], 'source_picture' : big_table['source_of_picture'][i],
                       'source_video': big_table['source_of_video'][i],
                       'len_video': big_table['len_video'][i]
                       })

In [50]:
header = ['message_post','id_post', 'time_created', 'hours_time_created', 'total_comments', 'total_of_reactions', 'reactions_like', 'reactions_love', 'reactions_wow'
          , 'reactions_haha',  "reactions_sad",  'reactions_angry', 'total_share', 'comments_details',
          'source_picture', 'source_video', 'len_video']

In [51]:
import csv
with open('data.csv','w', encoding='utf8') as handler:
    writer = csv.DictWriter(handler, fieldnames = header)
    writer.writeheader()
    writer.writerows(data_table)

In [52]:
for i in user_id_comment_set:
    print(i)

100016020637076
100022205889373
100008237892612
100002063422905
100006221594749
100005895515672
100036052644223
100006427000308
100010074492831
100033641495287
100003194471739
100007902755416
100010715224027
100010068193647
571495154
100028921999221
100004676025308
100033484025713
100032923447208
100033481483153
100022347820919
100031037400419
100007137946715
100034750493322
100007992400379
100005756288662
100003511993316
100076796252973
100025122667925
100071929246644
100026039470067
100024026661235
100006856531980
100007257754830
100009187442790
100007130407481
100040713137168
100056049884670
100009702404151
100075160842811
100013824019523
100076875781002
100003009079459
100087459398066
100015223572749
100003820568374
100022429271425
100010427084498
100036210196434
100076170917125
100005746053733
100075943951355
100056115011671
100004603487533
1175979190
100033075002001
100030747582126
100055928888980
100063528683509
100073072916532
100079497190516
100010730493122
100005265558349
100

In [53]:
#shell này chạy mất hơn 1h30p (gần 100 phút), lấy tát cả thông tin về user bình luận
fields_user_data = 'birthday,education,gender,hometown,first_name'#checkpoint
list_data_user = []
access_token_rack = 'EAABwzLixnjYBO9OzJc6pFWVUC2ZBJLn47cSHXH5Mfl4e09TZCe9CBlVDIuUI6WK0TlZCkpgCuoPrBBD7wojBwUaw4GZAwwLZCagAyrxyzgyIVRTmXxs9bqU2eauCZANGXqJLqbe0UxIxNt8WvSegpW9XrdXzxZCThSS5gRJiBL1MB64xSe1qv1uU16fmke0K1TMug03x0ZB9b3EB3nKpEd55kpFVsOIZD'

for i in user_id_comment_set:
     user_data = s.get('https://graph.facebook.com/'+i+'?fields='+fields_user_data+'&access_token='+access_token_rack)
     list_data_user.append(user_data.json())
     print(user_data.json())
#cho list_data_user vao file user_data_js.json    
with open('user_data_js.json','w') as handler:
    json.dump(list_data_user,handler)  

{'birthday': '06/23', 'gender': 'male', 'hometown': {'id': '106530866047287', 'name': 'Phu Tho, Phú Thọ, Vietnam'}, 'first_name': 'Nguyễn Tùng', 'id': '100016020637076'}
{'gender': 'male', 'first_name': 'Thành', 'id': '100022205889373'}
{'gender': 'male', 'first_name': 'Nguyễn', 'id': '100008237892612'}
{'education': [{'classes': [{'id': '189663084388553', 'name': '9A10', 'with': [{'name': 'Trung Đức Phan', 'id': '1830026538'}]}], 'school': {'id': '137465306291448', 'name': 'THCS nguyễn trường tộ'}, 'type': 'High School', 'id': '142360572509389'}, {'school': {'id': '105865386169931', 'name': 'THPT Bán Công Đống Đa -> THPT Hoàng Cầu - Đống Đa'}, 'type': 'High School', 'id': '1064463046965799'}], 'gender': 'male', 'hometown': {'id': '106388046062960', 'name': 'Hanoi, Vietnam'}, 'first_name': 'Phếu', 'id': '100002063422905'}
{'education': [{'school': {'id': '196046563782791', 'name': 'Học viện Tài Chính (Academy Of Finance)'}, 'type': 'College', 'id': '1927907870759914'}, {'school': {'id'